In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

## Data Loading and Preprocessing

In [ ]:
df = pd.read_csv('8taijiquan.csv')


In [ ]:
df.head()


In [ ]:
# Get unique values in the 'class' column
unique_classes = df['class'].unique()

# Display the unique values
print("Unique Classes:", unique_classes)

In [ ]:
# Check the distribution of classes
print(df['class'].value_counts())

In [ ]:
df[df['class']=='Horse Stance']


In [ ]:
print(df.dtypes)


## Histogram

In [ ]:
sns.set_palette('pastel')

plt.figure(figsize=(14, 6))

# Calculate the class frequencies and sort by frequency
class_counts = df['class'].value_counts().sort_values(ascending=False)

# Reorder the DataFrame based on class_counts
df_sorted = df[df['class'].isin(class_counts.index)]

# Plot the countplot with the sorted DataFrame
sns.countplot(data=df_sorted, x='class', palette='pastel', order=class_counts.index)

plt.xlabel('Stance', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.title('Distribution of Taijiquan', fontsize=16)

plt.xticks(rotation=45, ha='right')

for p in plt.gca().patches:
    plt.gca().text(p.get_x() + p.get_width() / 2., p.get_height(), f'{int(p.get_height())}',
                   ha='center', va='bottom', fontsize=12, color='black')

sns.set_style("whitegrid")
sns.despine()

plt.show()

##  Training, validation, and testing sets

In [ ]:
# Split the data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(
    df.drop('class', axis=1),  # Features (X)
    df['class'],                # Target variable (y)
    test_size=0.3,              # Percentage of data for the validation set
    random_state=42,            # Random state
    stratify=df['class']         # Class distribution in the splits
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp,                     # Features (X) after the first split
    y_temp,                     # Target variable (y) after the first split
    test_size=0.5,              # Percentage of data for the test set (relative to X_temp)
    random_state=42,            # Random state
    stratify=y_temp              # Class distribution in the splits
)


## Landmark-based Augmentation

In [ ]:
# Function to apply landmark-based augmentation
def apply_landmark_augmentation(landmarks, angle_range=(-10, 10), scale_range=(0.9, 1.1)):
    augmented_landmarks = landmarks.copy()
    num_landmarks = landmarks.shape[0]

    # Apply augmentation to each landmark
    for i in range(num_landmarks):
        angle = np.random.uniform(angle_range[0], angle_range[1])
        rotation_matrix = np.array([[np.cos(np.radians(angle)), -np.sin(np.radians(angle))],
                                    [np.sin(np.radians(angle)), np.cos(np.radians(angle))]])
        augmented_landmarks[i, :2] = np.dot(augmented_landmarks[i, :2], rotation_matrix.T)
        scale_factor = np.random.uniform(scale_range[0], scale_range[1])
        augmented_landmarks[i, :2] *= scale_factor

    return augmented_landmarks.flatten()

In [ ]:
# Apply landmark-based augmentation to X_train
X_train_augmented = []
for index, row in X_train.iterrows():
    landmarks = np.array(row).reshape(-1, 4)  
    augmented_landmarks = apply_landmark_augmentation(landmarks)
    X_train_augmented.append(augmented_landmarks)

X_train_augmented = pd.DataFrame(X_train_augmented, columns=X_train.columns)


## Model Definition and Training

In [ ]:
# Encode class labels to numerical values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
# Define the FNN model with modifications
def create_fnn_model(input_shape):
    model = Sequential()
    model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.0001), input_shape=input_shape))
    model.add(Dropout(0.5))  # Adjust dropout rate
    model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
    model.add(Dropout(0.5))  # Adjust dropout rate
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))

    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create and train the FNN model with early stopping
input_shape_fnn = (X_train_augmented.shape[1],)
fnn_model = create_fnn_model(input_shape_fnn)

# Number of training sessions
num_training_sessions = 2  # Adjust as needed

# List to store historical performance
historical_performance = []

for session in range(num_training_sessions):
    print(f"Training Session: {session + 1}/{num_training_sessions}")

    # Train the FNN model and record history
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = fnn_model.fit(X_train_augmented, y_train_encoded, epochs=10, batch_size=32, validation_data=(X_val, y_val_encoded), callbacks=[early_stopping])

    # Append the history to the list
    historical_performance.append(history.history)

# Access historical performance
for session, history in enumerate(historical_performance):
    print(f"\nTraining Session: {session + 1}/{num_training_sessions}")
    print(f"Final Training Loss: {history['loss'][-1]}")
    print(f"Final Validation Loss: {history['val_loss'][-1]}")
    print(f"Final Training Accuracy: {history['accuracy'][-1]}")
    print(f"Final Validation Accuracy: {history['val_accuracy'][-1]}")

    # Plot loss over epochs
    plt.figure(figsize=(10, 6))
    plt.plot(history['loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f"Training Session: {session + 1}")
    plt.legend()
    plt.show()

    # Plot accuracy over epochs
    plt.figure(figsize=(10, 6))
    plt.plot(history['accuracy'], label='Training Accuracy')
    plt.plot(history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title(f"Training Session: {session + 1}")
    plt.legend()
    plt.show()

## Model Evaluation and Testing

In [ ]:
# Evaluate the FNN model on the test set
test_loss_fnn, test_accuracy_fnn = fnn_model.evaluate(X_test, y_test_encoded)
print(f'Test Accuracy (FNN): {test_accuracy_fnn}')


In [ ]:
# Confusion Matrix for FNN
y_pred_fnn = fnn_model.predict(X_test)
y_pred_classes_fnn = np.argmax(y_pred_fnn, axis=1)
conf_matrix_fnn = confusion_matrix(y_test_encoded, y_pred_classes_fnn)
print(conf_matrix_fnn)
sns.heatmap(conf_matrix_fnn, annot=True, fmt='d', cmap='Blues', cbar=True)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (FNN)')
plt.show()

# Classification Report for FNN
class_report_fnn = classification_report(y_test_encoded, y_pred_classes_fnn)
print('Classification Report (FNN):')
print(class_report_fnn)


# K-Fold Cross Validation

In [ ]:
# K-Fold Cross Validation
X_train_augmented = np.array(X_train_augmented)
y_train_encoded = np.array(y_train_encoded)

# Define the number of folds
n_splits = 5  

# Initialize StratifiedKFold
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Lists to store results for each fold
all_train_accuracy = []
all_val_accuracy = []

# Loop through the folds
for fold_num, (train_index, val_index) in enumerate(kf.split(X_train_augmented, y_train_encoded), 1):
    X_train_fold, X_val_fold = X_train_augmented[train_index], X_train_augmented[val_index]
    y_train_fold, y_val_fold = y_train_encoded[train_index], y_train_encoded[val_index]

    # Create and train the FNN model for each fold
    fnn_model = create_fnn_model(input_shape_fnn)
    history = fnn_model.fit(
        X_train_fold, y_train_fold,
        epochs=20, batch_size=32,
        validation_data=(X_val_fold, y_val_fold),
        callbacks=[early_stopping]
    )

    # Evaluate the model on the training and validation data for each fold
    _, train_accuracy = fnn_model.evaluate(X_train_fold, y_train_fold, verbose=0)
    _, val_accuracy = fnn_model.evaluate(X_val_fold, y_val_fold, verbose=0)

    print(f"Fold {fold_num}: Training Accuracy = {train_accuracy:.4f}, Validation Accuracy = {val_accuracy:.4f}")

    # Store the training and validation accuracy for each fold
    all_train_accuracy.append(train_accuracy)
    all_val_accuracy.append(val_accuracy)

# Calculate the average training and validation accuracy over all folds
avg_train_accuracy = np.mean(all_train_accuracy)
avg_val_accuracy = np.mean(all_val_accuracy)

print(f"\nAverage Training Accuracy Across Folds = {avg_train_accuracy:.4f}")
print(f"Average Validation Accuracy Across Folds = {avg_val_accuracy:.4f}")


## Percentage of Keypoints

In [ ]:

# Define a function to calculate accuracy for a single keypoint
def calculate_keypoint_accuracy(predicted_keypoint, ground_truth_keypoint, threshold):
    distance = np.linalg.norm(np.array(predicted_keypoint) - np.array(ground_truth_keypoint))
    return int(distance < threshold)

# Define a function to calculate accuracy for all 33 landmarks
def calculate_overall_accuracy(predicted_landmarks, ground_truth_landmarks, threshold):
    num_landmarks = len(predicted_landmarks)
    accuracies = [calculate_keypoint_accuracy(predicted_landmarks[i], ground_truth_landmarks[i], threshold) for i in range(num_landmarks)]
    percentage_accurate = sum(accuracies) / num_landmarks * 100
    return percentage_accurate

# Usage
threshold = 0.5  # Define your accuracy threshold
predicted_landmarks = y_pred_classes_fnn  
ground_truth_landmarks = y_test_encoded    

accuracy = calculate_overall_accuracy(predicted_landmarks, ground_truth_landmarks, threshold)
print(f"Overall Accuracy: {accuracy:.2f}%")


## Statistical Testing: Paired T-test

In [ ]:
from scipy.stats import ttest_rel

# Combine the training and validation accuracies for each fold into two arrays
all_train_accuracy = [history['accuracy'][-1] for history in historical_performance]
all_val_accuracy = [history['val_accuracy'][-1] for history in historical_performance]

# Perform a paired sample t-test
statistic, p_value = ttest_rel(all_train_accuracy, all_val_accuracy)

# Print the results
print(f't-statistic: {statistic}')
print(f'P-value: {p_value}')

# Interpret the results
alpha = 0.05  # Significance level
if p_value < alpha:
    print("Reject the null hypothesis. There is a significant difference between training and validation accuracies.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference between training and validation accuracies.")


t-statistic: -11.208570000845578

The t-statistic is a measure of how far the sample mean (in this case, the difference between training and validation accuracies) is from the population mean, in terms of standard errors. A larger absolute t-statistic suggests a greater difference between the sample means.

In your case, the negative sign indicates that, on average, the validation accuracies are lower than the training accuracies.

P-value: 0.00036078544601951236

The p-value is the probability of observing a t-statistic as extreme as the one calculated, assuming that the null hypothesis is true. A lower p-value indicates stronger evidence against the null hypothesis.

In your case, the very low p-value (0.00036) suggests that the observed difference in accuracies is unlikely to have occurred by random chance alone.

Interpretation:

The phrase "Reject the null hypothesis" means that there is enough evidence to conclude that the training and validation accuracies are significantly different.

The phrase "There is a significant difference between training and validation accuracies" reinforces the rejection of the null hypothesis, indicating that the observed difference is not likely due to random fluctuations.

## Model Deployment, Inference and HPE

In [ ]:
import pickle
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
# Save the trained FNN model
fnn_model.save('saved_model/fnn_model')

# Save the label encoder for later use
with open('saved_model/label_encoder.pkl', 'wb') as le_file:
    pickle.dump(label_encoder, le_file)


In [ ]:
# Load the saved model
loaded_model = tf.keras.models.load_model('saved_model/fnn_model')

# Load the label encoder
with open('saved_model/label_encoder.pkl', 'rb') as le_file:
    loaded_label_encoder = pickle.load(le_file)


In [ ]:
# Initialize BlazePose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
)

# Start capturing video from the camera
cap = cv2.VideoCapture(0)  # 0 for the default camera, adjust if necessary
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1200)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 500)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with BlazePose
    results = pose.process(frame_rgb)

    # Recolor image back to BGR for rendering
    frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

    # Detect Taijiquan Stances (class)
    if results.pose_landmarks:
        # Extract Pose landmarks
        pose_landmarks = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose_landmarks]).flatten())

        # Make Detections
        X = pd.DataFrame([pose_row])

        # Convert X to numpy array
        input_data = X.to_numpy().astype(np.float32)

        # Make predictions using the Keras model
        predictions = loaded_model.predict(input_data)
        body_language_class = np.argmax(predictions)
        body_language_prob = predictions[0]

        print(body_language_class, body_language_prob)

        # Display Probability
        cv2.putText(frame, 'PROB', (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(frame, str(round(body_language_prob[body_language_class], 2)),
                    (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1, cv2.LINE_AA)

        # Display detected class
        cv2.putText(frame, f'CLASS: {loaded_label_encoder.classes_[body_language_class]}', (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1, cv2.LINE_AA)

        # Draw pose landmarks
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
        )

    cv2.imshow('Pose Detection', frame)

    # Check for exit key (q)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [ ]:
# Initialize BlazePose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
)

# Start capturing video from the camera
cap = cv2.VideoCapture(0)  # 0 for the default camera, adjust if necessary
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1200)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 800)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with BlazePose
    results = pose.process(frame_rgb)

    # Recolor image back to BGR for rendering
    frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

    # Detect Taijiquan Stances (class)
    if results.pose_landmarks:
        # Extract Pose landmarks
        pose_landmarks = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose_landmarks]).flatten())

        # Make Detections
        X = pd.DataFrame([pose_row])

        # Convert X to numpy array
        input_data = X.to_numpy().astype(np.float32)

        # Run inference using the loaded FNN model
        # Make predictions using the Keras model
        predictions = loaded_model.predict(input_data)
        body_language_class = np.argmax(predictions)
        body_language_prob = predictions[0] if len(predictions.shape) == 2 else predictions

        #print(body_language_class, body_language_prob)

        # Convert landmark coordinates to integers
        landmarks_as_pixels = np.array([(int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])) for landmark in results.pose_landmarks.landmark])

        # Calculate bounding rectangle
        bbox_c = cv2.boundingRect(landmarks_as_pixels)

        # Draw the bounding box
        cv2.rectangle(frame, (int(bbox_c[0]), int(bbox_c[1])), (int(bbox_c[0] + bbox_c[2]), int(bbox_c[1] + bbox_c[3])), (0, 255, 0), 2)

        # Display Probability
        cv2.putText(frame, 'PROB', (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, str(round(body_language_prob[0, body_language_class], 2)),
                    (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Display detected class
        cv2.putText(frame, f'CLASS: {body_language_class}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Draw pose landmarks
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(0, 191, 255), thickness=2, circle_radius=2)
        )

        # Add class label on the bounding box
        cv2.putText(frame, f'CLASS: {body_language_class}', (int(bbox_c[0]), int(bbox_c[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Pose Detection', frame)

    # Check for exit key (q)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()